# Expose MCP Text Utilities Server via Azure API Management

This notebook registers an **already-deployed** MCP server (running in Azure Container Apps) as a
Streamable-HTTP MCP API inside an existing Azure API Management instance, then validates the
end-to-end flow by calling one tool through the APIM gateway.

### What this notebook does
1. Verifies your Azure CLI session.
2. Retrieves the APIM gateway URL.
3. Deploys `apim/mcp-api.bicep` – creates the APIM backend + MCP API + policy.
4. Tests the APIM endpoint: lists tools and calls `generate_uuid`.

### Prerequisites
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed and signed in.
- Contributor access on the resource group that contains your APIM instance.
- Python packages: `mcp`, `nest_asyncio` (installed in the cell below).

▶️ Click **Run All** to execute all steps, or run them **Step by Step**.

### 0️⃣ Initialize notebook variables

Update the values below to match your deployment.

In [ ]:
import os, sys, json, subprocess

# ── Existing APIM ──────────────────────────────────────────────────────────────
apim_resource_name     = "apim-bc2yajbbxycfw"
resource_group_name    = "rg-citadel-hub-dev-1"

# ── Already-deployed MCP server ────────────────────────────────────────────────
# Full URL of the /mcp endpoint on your Container App
mcp_server_url = "https://ca-agent-cpraudttz2w5u.yellowfield-eea789b1.swedencentral.azurecontainerapps.io/mcp"

# Path segment that will be used inside the APIM gateway
# → final MCP URL will be: <apim_gateway_url>/<mcp_path>/mcp
mcp_path = "text-utils"

# ── Container App (EasyAuth must be disabled so APIM can reach the backend) ───────────────
# Derive the Container App name from the FQDN in mcp_server_url.
try:
    _ca_fqdn = mcp_server_url.split("//", 1)[1].split("/")[0]  # e.g. ca-agent-xxx.env.region.azurecontainerapps.io
    container_app_name = _ca_fqdn.split(".")[0]               # e.g. ca-agent-xxx
except (IndexError, ValueError) as _e:
    raise ValueError(
        f"Cannot parse Container App name from mcp_server_url='{mcp_server_url}'. "
        "Expected format: https://<ca-name>.<env>.<region>.azurecontainerapps.io/mcp"
    ) from _e

# Look up the resource group that contains the Container App.
_rg_result = subprocess.run(
    [
        "az", "resource", "list",
        "--name",          container_app_name,
        "--resource-type", "Microsoft.App/containerApps",
        "--query",         "[0].resourceGroup",
        "-o",              "tsv",
    ],
    capture_output=True, text=True
)
container_app_rg = _rg_result.stdout.strip()
if not container_app_rg:
    raise RuntimeError(
        f"Could not find a Container App named '{container_app_name}'. "
        "Check that mcp_server_url is correct and that you are signed into the right subscription."
    )

# ── Bicep template (relative to repo root) ─────────────────────────────────────
bicep_file = os.path.join(os.path.dirname(os.path.abspath('')), 'apim', 'mcp-api.bicep')
if not os.path.exists(bicep_file):
    # Fallback: assume notebook is run from repo root
    bicep_file = os.path.join('apim', 'mcp-api.bicep')

print(f"✅ Variables initialised")
print(f"   APIM:          {apim_resource_name}")
print(f"   Resource Group: {resource_group_name}")
print(f"   MCP server URL: {mcp_server_url}")
print(f"   APIM path:      {mcp_path}/mcp")
print(f"   Container App:  {container_app_name} (RG: {container_app_rg})")
print(f"   Bicep file:     {bicep_file}")

### 1️⃣ Verify the Azure CLI and the connected Azure subscription

In [ ]:
result = subprocess.run(["az", "account", "show", "-o", "json"],
                        capture_output=True, text=True)
if result.returncode != 0:
    raise RuntimeError(f"❌ az account show failed – are you signed in?\n{result.stderr}")

account = json.loads(result.stdout)
current_user    = account["user"]["name"]
tenant_id       = account["tenantId"]
subscription_id = account["id"]

print(f"✅ Signed in")
print(f"   User:           {current_user}")
print(f"   Tenant ID:      {tenant_id}")
print(f"   Subscription:   {subscription_id}")

### 2️⃣ Retrieve the APIM gateway URL

In [ ]:
result = subprocess.run(
    ["az", "apim", "show",
     "--name", apim_resource_name,
     "--resource-group", resource_group_name,
     "--query", "gatewayUrl",
     "-o", "tsv"],
    capture_output=True, text=True
)
if result.returncode != 0:
    raise RuntimeError(f"❌ Failed to retrieve APIM gateway URL:\n{result.stderr}")

apim_gateway_url = result.stdout.strip()
apim_mcp_url     = f"{apim_gateway_url}/{mcp_path}/mcp"

print(f"✅ APIM gateway URL:  {apim_gateway_url}")
print(f"   MCP endpoint via APIM: {apim_mcp_url}")

### 2️⃣b Disable EasyAuth on the Container App

The Container App was provisioned with Azure AD EasyAuth enabled
(`unauthenticatedClientAction: Return401`). APIM calls the backend without a user token,
so EasyAuth must be disabled before APIM can proxy requests to the server.
APIM becomes the access-control layer (subscription key or policy).

> 💡 To re-enable EasyAuth later: `az containerapp auth update --name <name> --resource-group <rg> --enabled true`

In [ ]:
print(f"⏳ Disabling EasyAuth on Container App '{container_app_name}' in RG '{container_app_rg}' …")
result = subprocess.run(
    [
        "az", "containerapp", "auth", "update",
        "--name",           container_app_name,
        "--resource-group", container_app_rg,
        "--enabled",        "false",
    ],
    capture_output=True, text=True
)
if result.returncode != 0:
    raise RuntimeError(
        f"❌ Failed to disable EasyAuth on '{container_app_name}':\n{result.stderr}"
    )
print(f"✅ EasyAuth disabled — APIM can now reach the backend unauthenticated")

### 3️⃣ Deploy the MCP API into APIM using Bicep

The Bicep template (`apim/mcp-api.bicep`) creates:
- An APIM **Backend** pointing to your Container App `/mcp` endpoint.
- An APIM **API** of `type: mcp` with Streamable-HTTP transport.
- A pass-through **Policy**.


In [ ]:
deployment_name = "mcp-text-utils-apim"

cmd = [
    "az", "deployment", "group", "create",
    "--name",           deployment_name,
    "--resource-group", resource_group_name,
    "--template-file",  bicep_file,
    "--parameters",     f"apimServiceName={apim_resource_name}",
    "--parameters",     f"mcpBackendUrl={mcp_server_url}",
    "--parameters",     f"mcpPath={mcp_path}",
    "-o", "json"
]

print(f"⏳ Deploying '{deployment_name}' to resource group '{resource_group_name}' …")
result = subprocess.run(cmd, capture_output=True, text=True)

if result.returncode != 0:
    raise RuntimeError(f"❌ Deployment failed:\n{result.stderr}")

deployment_output = json.loads(result.stdout)
outputs = deployment_output.get("properties", {}).get("outputs", {})

gateway_url_out = outputs.get("apimGatewayUrl", {}).get("value", apim_gateway_url)
mcp_api_path    = outputs.get("mcpApiPath",   {}).get("value", f"{mcp_path}/mcp")

apim_mcp_url = f"{gateway_url_out}/{mcp_path}/mcp"

print(f"✅ Deployment succeeded")
print(f"   APIM Gateway:   {gateway_url_out}")
print(f"   MCP API path:   {mcp_api_path}")
print(f"   MCP URL via APIM: {apim_mcp_url}")

### 4️⃣ Install Python dependencies

In [ ]:
subprocess.run([sys.executable, "-m", "pip", "install", "--quiet", "mcp", "nest_asyncio"], check=True)
print("✅ Dependencies ready")

### 🧪 Test – List tools exposed by the MCP server through APIM

In [ ]:
import asyncio
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client
import nest_asyncio
nest_asyncio.apply()

async def list_tools(server_url):
    streams_ctx = streamablehttp_client(server_url)
    streams = await streams_ctx.__aenter__()
    session_ctx = ClientSession(streams[0], streams[1])
    session = await session_ctx.__aenter__()
    try:
        await session.initialize()
        response = await session.list_tools()
        print(f"✅ Connected to {server_url}")
        print("⚙️  Tools available:")
        for tool in response.tools:
            print(f"   • {tool.name}: {tool.description}")
        return response.tools
    except Exception as e:
        print(f"❌ Error listing tools: {e}")
        return []
    finally:
        await session_ctx.__aexit__(None, None, None)
        await streams_ctx.__aexit__(None, None, None)

tools = asyncio.run(list_tools(apim_mcp_url))

### 🧪 Test – Call the `generate_uuid` tool through APIM

This tool requires no arguments and returns a freshly generated UUID v4 — the simplest possible
end-to-end smoke test.

In [ ]:
async def call_generate_uuid(server_url):
    streams_ctx = streamablehttp_client(server_url)
    streams = await streams_ctx.__aenter__()
    session_ctx = ClientSession(streams[0], streams[1])
    session = await session_ctx.__aenter__()
    try:
        await session.initialize()
        result = await session.call_tool("generate_uuid", {})
        uuid_value = result.content[0].text if result.content else "(empty response)"
        print(f"✅ generate_uuid → {uuid_value}")
        return uuid_value
    except Exception as e:
        print(f"❌ Error calling generate_uuid: {e}")
    finally:
        await session_ctx.__aexit__(None, None, None)
        await streams_ctx.__aexit__(None, None, None)

asyncio.run(call_generate_uuid(apim_mcp_url))

### ℹ️ (Optional) Use the MCP Inspector for interactive testing

1. Run `npx @modelcontextprotocol/inspector` in a terminal.
2. Open the printed URL in a browser.
3. Set transport type to **Streamable HTTP**.
4. Set the URL to the value of `apim_mcp_url` printed above.
5. Click **Connect** and explore the **Tools** tab.